In [3]:
import os # to log to specific positions
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy # average reward over episodes. plus std deviation
from datetime import datetime


In [4]:
environment_name = 'CartPole-v0'
env = gym.make(environment_name)



In [5]:
episodes = 5
for episode in range(1, episodes +1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()



Episode:1 Score:17.0
Episode:2 Score:28.0
Episode:3 Score:14.0
Episode:4 Score:14.0
Episode:5 Score:32.0


In [6]:
log_path = os.path.join(os.getcwd(), 'logs')

env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

model.learn(total_timesteps=20_000)

PPO_Path = os.path.join(os.getcwd(), 'models', 'PPO_model_' + environment_name + '_' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))

model.save(PPO_Path)

Using cuda device
Logging to d:\git\rl-test\nicholas-renotte\3h-rl\logs\PPO_2
-----------------------------
| time/              |      |
|    fps             | 609  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 486         |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009139267 |
|    clip_fraction        | 0.0896      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00246     |
|    learning_rate        | 0.0003      |
|    loss                 | 7.04        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0139     |
|    value_loss           | 51.2        